<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim

Dataset utilizado puede ser encontrado en kaggle como Donald Trump Rally Speeches
 https://www.kaggle.com/datasets/christianlillelund/donald-trumps-rallies



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [6]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

import nltk
nltk.download('stopwords')
nltk.download('wordnet')

from sklearn.manifold import TSNE
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import requests

import platform
from keras.preprocessing.text import text_to_word_sequence
from gensim.models.callbacks import CallbackAny2Vec                   
import plotly.graph_objects as go
import plotly.express as px



### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [13]:
url = "https://raw.githubusercontent.com/Oks11111/NLP_TP/main/3_archive.zip"

response = requests.get(url)
if response.status_code == 200:
    with open("3_archive.zip", "wb") as file:
        file.write(response.content)
    print("File downloaded successfully.")
else:
    print("Error downloading the file. Status code:", response.status_code)


Error downloading the file. Status code: 404


In [14]:

dataset_folder = './3_archive'
zip_file = '3_archive.zip'

if not os.path.exists(dataset_folder):
    if not os.path.exists(zip_file):
        if platform.system() == 'Windows':
            !curl -LJO https://raw.githubusercontent.com/Oks11111/NLP_TP/main/3_archive.zip
        else:
            !wget https://raw.githubusercontent.com/Oks11111/NLP_TP/main/3_archive.zip
    if os.path.exists(zip_file):
        !unzip -q 3_archive.zip
    else:
        print("Error: Failed to download the dataset zip file.")
else:
    print("El dataset ya se encuentra descargado")



--2023-06-07 01:00:13--  https://raw.githubusercontent.com/Oks11111/NLP_TP/main/3_archive.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 737172 (720K) [application/zip]
Saving to: ‘3_archive.zip’

3_archive.zip       100%[===================>] 719.89K  --.-KB/s    in 0.009s  

2023-06-07 01:00:14 (79.1 MB/s) - ‘3_archive.zip’ saved [737172/737172]



In [16]:
! ls

3_archive.zip			MinneapolisOct10_2019.txt
BattleCreekDec19_2019.txt	MosineeSep17_2020.txt
BemidjiSep18_2020.txt		NewHampshireAug15_2019.txt
CharlestonFeb28_2020.txt	NewHampshireAug28_2020.txt
CharlotteMar2_2020.txt		NewHampshireFeb10_2020.txt
CincinnatiAug1_2019.txt		NewMexicoSep16_2019.txt
ColoradorSpringsFeb20_2020.txt	OhioSep21_2020.txt
DallasOct17_2019.txt		PhoenixFeb19_2020.txt
DesMoinesJan30_2020.txt		PittsburghSep22_2020.txt
FayettevilleSep19_2020.txt	sample_data
FayettevilleSep9_2019.txt	songs_dataset
FreelandSep10_2020.txt		songs_dataset.zip
GreenvilleJul17_2019.txt	TexasSep23_2019.txt
HendersonSep13_2020.txt		ToledoJan9_2020.txt
HersheyDec10_2019.txt		TulsaJun20_2020.txt
LasVegasFeb21_2020.txt		TupeloNov1_2019.txt
LatrobeSep3_2020.txt		WildwoodJan28_2020.txt
LexingtonNov4_2019.txt		Winston-SalemSep8_2020.txt
MilwaukeeJan14_2020.txt		YumaAug18_2020.txt
MindenSep12_2020.txt


In [44]:

folder_path = './'  
files = os.listdir(folder_path)

data = []

for file in files:
    if file.endswith('.txt'):
        file_path = os.path.join(folder_path, file)
        with open(file_path, 'r') as f:
            content = f.read().strip()  
            data.append(content)

df = pd.DataFrame(data, columns=['text'])
df.head()


,text
0,"Hello Colorado. We love Colorado, most beautif..."
1,Thank you. Thank you. Thank you. All I can say...
2,Thank you very much everybody. Thank you. Wow....
3,Thank you. Thank you. Thank you to Vice Presid...
4,"Thank you very much, Phoenix. We love to be ba..."


In [45]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 35


### 1 - Preprocesamiento

In [46]:
sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [47]:
# Demos un vistazo
sentence_tokens[:2]

[['hello',
  'colorado',
  'we',
  'love',
  'colorado',
  'most',
  'beautiful',
  'place',
  'and',
  "i'm",
  'thrilled',
  'to',
  'be',
  'back',
  'in',
  'colorado',
  'springs',
  "i've",
  'spent',
  'a',
  'lot',
  'of',
  'time',
  'here',
  'over',
  'the',
  'years',
  'beautiful',
  'place',
  'beneath',
  'the',
  'majestic',
  'peaks',
  "that's",
  'true',
  'of',
  'the',
  'rocky',
  'mountains',
  'with',
  'thousands',
  'of',
  'terrific',
  'hardworking',
  'great',
  'american',
  'patriots',
  'with',
  'your',
  'help',
  'this',
  'november',
  'we',
  'are',
  'going',
  'to',
  'defeat',
  'the',
  'radical',
  'democrats',
  'and',
  'we',
  'are',
  'going',
  'to',
  'win',
  'colorado',
  'in',
  'a',
  'landslide',
  'and',
  "you're",
  'going',
  'to',
  'help',
  'us',
  'get',
  'cory',
  'gardner',
  'across',
  'that',
  'line',
  'because',
  "he's",
  'been',
  'with',
  'us',
  '100',
  'there',
  'was',
  'no',
  'waiver',
  "he's",
  'been',

### 2 - Crear los vectores (word2vec)

In [48]:
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [56]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=100,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [57]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [58]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 35


In [59]:
# Cantidad de palabras encontradas en el corpus
print("Cantidad de palabras distintas en el corpus:", len(w2v_model.wv.key_to_index))


Cantidad de palabras distintas en el corpus: 3659


### 3 - Entrenar el modelo generador

In [60]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=50,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 2955274.25
Loss after epoch 1: 2193211.25
Loss after epoch 2: 2064366.5
Loss after epoch 3: 1984755.0
Loss after epoch 4: 1904820.0
Loss after epoch 5: 1878709.0
Loss after epoch 6: 1858615.0
Loss after epoch 7: 1845739.0
Loss after epoch 8: 1761922.0
Loss after epoch 9: 1744932.0
Loss after epoch 10: 1736980.0
Loss after epoch 11: 1727800.0
Loss after epoch 12: 1720608.0
Loss after epoch 13: 1711448.0
Loss after epoch 14: 1710064.0
Loss after epoch 15: 1700882.0
Loss after epoch 16: 1696416.0
Loss after epoch 17: 1693282.0
Loss after epoch 18: 1714588.0
Loss after epoch 19: 1712764.0
Loss after epoch 20: 1705104.0
Loss after epoch 21: 1703056.0
Loss after epoch 22: 1699796.0
Loss after epoch 23: 1692644.0
Loss after epoch 24: 1686632.0
Loss after epoch 25: 1683692.0
Loss after epoch 26: 1679780.0
Loss after epoch 27: 1678752.0
Loss after epoch 28: 1679624.0
Loss after epoch 29: 1679204.0
Loss after epoch 30: 1673248.0
Loss after epoch 31: 1673180.0
Loss after epoch

(12835505, 18584600)

### 4 - Ensayar

In [61]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["never"], topn=10)

[('ever', 0.5005500316619873),
 ("nobody's", 0.4885869026184082),
 ('anything', 0.4763788878917694),
 ('before', 0.47092702984809875),
 ('nobody', 0.4663262367248535),
 ('ronald', 0.462356299161911),
 ('always', 0.45965415239334106),
 ('maintain', 0.4344194829463959),
 ('pinocchio', 0.4338793158531189),
 ('it…', 0.4337335228919983)]

In [62]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["love"], topn=10)

[('7', 0.07590439915657043),
 ('hundred', 0.0686504915356636),
 ('mini', 0.06808686256408691),
 ('invest', 0.06495071947574615),
 ('vaccine', 0.061710283160209656),
 ('4', 0.056252919137477875),
 ('gain', 0.053372062742710114),
 ('fraud', 0.048461753875017166),
 ('illegal', 0.045869436115026474),
 ('33', 0.04318276792764664)]

In [63]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["four"], topn=10)

[('three', 0.5608438849449158),
 ('five', 0.5569336414337158),
 ('six', 0.5406760573387146),
 ('seven', 0.49826210737228394),
 ('two', 0.4806632399559021),
 ('years', 0.48031479120254517),
 ('43', 0.4562414884567261),
 ('fortunately', 0.4548368752002716),
 ('thousand', 0.4538187086582184),
 ('51', 0.44495850801467896)]

In [64]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["money"], topn=5)

[('borrow', 0.5540812015533447),
 ('fortune', 0.467087060213089),
 ('car', 0.46600380539894104),
 ('billions', 0.4633437395095825),
 ('pump', 0.4623432159423828)]

In [66]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["believe"])

[('generals', 0.06878174841403961),
 ('planet', 0.05955793708562851),
 ('ukraine', 0.052293963730335236),
 ('mic', 0.04982615262269974),
 ('prisoners', 0.0389232262969017),
 ('nato', 0.03851529210805893),
 ('speaker', 0.0361144058406353),
 ('lottery', 0.026974037289619446),
 ('disinformation', 0.023768166080117226),
 ('asked', 0.022894661873579025)]

### 5 - Visualizar agrupación de vectores

In [67]:
                             

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [68]:
# Graficar los embedddings en 2D
x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

#### En el caso actual vemos muchas stopwords y las combinaciónes clasicas de palabras. Entre estas podemos rescatar algunas asociaciones. Entre ellas se resaltan las combinaciones de numeros sobre todo antonimos y sinonimos. Otras como:
#### greatest-word; bad-good-trump. Vamos a mejorar un poco el preproceso y veamos el resultado

### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.

#### Veamos que sucede si preprocesamos un poco el texto:

In [82]:
import numpy as np
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import plotly.graph_objects as go
import plotly.express as px
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Función para quitar stopwords y lematizar el texto
def preprocess_text(text):
    # Remover stopwords
    stop_words = set(stopwords.words('english'))
    text = [word for word in text if word.lower() not in stop_words]

    # Lematizar palabras
    lemmatizer = WordNetLemmatizer()
    lemmatized_text = [lemmatizer.lemmatize(word) for word in text]

    return lemmatized_text

# Obtener tokens de oraciones y aplicar preprocesamiento
preprocessed_tokens = [preprocess_text(sentence) for sentence in sentence_tokens]

# Entrenar el modelo generador de vectores
w2v_model.train(preprocessed_tokens,
                total_examples=w2v_model.corpus_count,
                epochs=40,
                compute_loss=True,
                callbacks=[callback()]
                )

# Función para reducir las dimensiones de los vectores utilizando t-SNE
def reduce_dimensions(model):
    num_dimensions = 2

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels



Loss after epoch 0: 258330.921875
Loss after epoch 1: 268165.265625
Loss after epoch 2: 300707.125
Loss after epoch 3: 248139.6875
Loss after epoch 4: 255382.75
Loss after epoch 5: 339061.0
Loss after epoch 6: 244855.375
Loss after epoch 7: 242330.125
Loss after epoch 8: 228557.25
Loss after epoch 9: 243157.0
Loss after epoch 10: 256311.25
Loss after epoch 11: 249356.25
Loss after epoch 12: 251444.5
Loss after epoch 13: 259569.75
Loss after epoch 14: 251569.25
Loss after epoch 15: 231234.25
Loss after epoch 16: 224027.75
Loss after epoch 17: 213136.0
Loss after epoch 18: 191410.0
Loss after epoch 19: 204202.0
Loss after epoch 20: 216817.0
Loss after epoch 21: 182812.5
Loss after epoch 22: 192445.5
Loss after epoch 23: 219972.5
Loss after epoch 24: 249450.5
Loss after epoch 25: 222473.5
Loss after epoch 26: 209001.5
Loss after epoch 27: 182997.0
Loss after epoch 28: 204188.5
Loss after epoch 29: 223730.0
Loss after epoch 30: 202424.0
Loss after epoch 31: 213567.0
Loss after epoch 32: 21

In [83]:
# Obtener las dimensiones reducidas y las etiquetas
x_vals, y_vals, labels = reduce_dimensions(w2v_model)

# Graficar los embeddings en 2D usando Plotly
MAX_WORDS = 200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab")

#### Algunas combinaciones de palabras: right-know; americans-american family-god; trade-deal; north-carolina

El preproceso limpió un poco el contenido pero no ha mejorado lo suficiente el dataset. Se puede observar que las combinaciones que se obtienen de un discurso politico son las esperadas. Ademas de las combinaciones y asociaciones de numericos, sinonimos, antonimos, combinaciones tambien se entreven las posturas politicas especificas. 